# Using Orion on multivariate input

In this notebook, we demonstrate how you can use multivariate time series in Orion. We will walk through the process using NASA's dataset, you can find the original data in [Telemanom](https://github.com/khundman/telemanom) github or directly from their [S3 bucket](https://s3-us-west-2.amazonaws.com/telemanom/data.zip).

## 1. Load the data

In the first step, we setup the environment and load the CSV that we want to process.

To do so, we need to import the `orion.data.load_signal` function and call it passing
the path to the CSV file.

In this case, we will be loading the `S-1.csv` file from inside the `data/multivariate` folder.

In [1]:
from orion.data import load_signal

signal_path = 'multivariate/S-1'

data = load_signal(signal_path)
data.head()

,timestamp,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
0,1222819200,-0.366359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1222840800,-0.394108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1222862400,0.403625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1222884000,-0.362759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1222905600,-0.370746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2. Detect anomalies using Orion

Once we have the data, let us try to use the LSTM pipeline to analyze it and search for anomalies.

In order to do so, we will have import the `Orion` class from `orion.core` and pass it
the loaded data and the path to the pipeline JSON that we want to use.

In this case, we will be using the `lstm_dynamic_threshold` pipeline from inside the `orion` folder. 

In addition, we setup the hyperparameters to correctly identify the signal I am trying to predict. In this case, dimension `0` is the signal value and such we set `target_column` to `0`. Note that `0` refers to the location of the channel rather than the name.

In [3]:
from orion import Orion

hyperparameters = {
    "mlprimitives.custom.timeseries_preprocessing.rolling_window_sequences#1": {
        'target_column': 0 
    },
    'keras.Sequential.LSTMTimeSeriesRegressor#1': {
        'epochs': 5,
        'verbose': True
    }
}

orion = Orion(
    pipeline='lstm_dynamic_threshold',
    hyperparameters=hyperparameters
)

orion.fit(data)

2021-09-28 23:11:54.644061: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-09-28 23:11:54.700416: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f8301da98d0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-09-28 23:11:54.700434: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version



Train on 7919 samples, validate on 1980 samples
Epoch 1/5
7919/7919 [==============================] - 33s 4ms/step - loss: 0.2097 - mse: 0.2097 - val_loss: 0.2673 - val_mse: 0.2673
Epoch 2/5
7919/7919 [==============================] - 30s 4ms/step - loss: 0.1997 - mse: 0.1997 - val_loss: 0.2557 - val_mse: 0.2557
Epoch 3/5
7919/7919 [==============================] - 31s 4ms/step - loss: 0.1980 - mse: 0.1980 - val_loss: 0.2581 - val_mse: 0.2581
Epoch 4/5
7919/7919 [==============================] - 31s 4ms/step - loss: 0.1950 - mse: 0.1950 - val_loss: 0.2702 - val_mse: 0.2702
Epoch 5/5
9899/9899 [==============================] - 12s 1ms/step


The output will be a ``pandas.DataFrame`` containing a table with the detected anomalies.

In [4]:
orion.detect(data)

9899/9899 [==============================] - 12s 1ms/step


,start,end,severity
0,1228219200,1229472000,0.792486


For reconstruction based pipelines, we need to specify the shape of the **input** and **target** sequences. For example, assume we are using the `lstm_autoencoder` pipeline, we set the hyperparameter values 

```python3
hyperparameters = {
    "mlprimitives.custom.timeseries_preprocessing.rolling_window_sequences#1": {
        'window_size': 100,
        'target_column': 0 
    },
    'keras.Sequential.LSTMSeq2Seq#1': {
        'epochs': 5,
        'verbose': True,
        'input_shape': [100, 25],
        'target_shape': [100, 1],
    }
}
```

where the shape of the input is dependent on 

1. `window_size` and 
2. the number of channels in the data.

similarly, the shape of the output is dependent on the `window_size`, and currently we are focusing on multivariate input and univariate output, therefore the target shape should always be [`window_size`, 1].

In [5]:
hyperparameters = {
    "mlprimitives.custom.timeseries_preprocessing.rolling_window_sequences#1": {
        "window_size": 100,
        "target_column": 0 
    },
    'keras.Sequential.LSTMSeq2Seq#1': {
        'epochs': 5,
        'verbose': True,
        'input_shape': [100, 25],
        'target_shape': [100, 1],
    }
}

orion = Orion(
    pipeline='lstm_autoencoder',
    hyperparameters=hyperparameters
)

orion.fit(data)

Train on 8039 samples, validate on 2010 samples
Epoch 1/5


2021-09-28 23:18:43.564815: E tensorflow/core/grappler/optimizers/dependency_optimizer.cc:697] Iteration = 0, topological sort failed with message: The graph couldn't be sorted in topological order.
2021-09-28 23:18:43.569190: E tensorflow/core/grappler/optimizers/dependency_optimizer.cc:697] Iteration = 1, topological sort failed with message: The graph couldn't be sorted in topological order.
2021-09-28 23:18:43.593575: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:533] model_pruner failed: Invalid argument: MutableGraphView::MutableGraphView error: node 'loss_1/reshape_1_loss/mean_squared_error/weighted_loss/concat' has self cycle fanin 'loss_1/reshape_1_loss/mean_squared_error/weighted_loss/concat'.
2021-09-28 23:18:43.640070: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:533] remapper failed: Invalid argument: MutableGraphView::MutableGraphView error: node 'loss_1/reshape_1_loss/mean_squared_error/weighted_loss/concat' has self cycle fanin 'loss_1/reshape_1_los

8000/8039 [============================>.] - ETA: 0s - loss: 0.2007 - mse: 0.2007

2021-09-28 23:18:53.709501: E tensorflow/core/grappler/optimizers/dependency_optimizer.cc:697] Iteration = 0, topological sort failed with message: The graph couldn't be sorted in topological order.
2021-09-28 23:18:53.710394: E tensorflow/core/grappler/optimizers/dependency_optimizer.cc:697] Iteration = 1, topological sort failed with message: The graph couldn't be sorted in topological order.
2021-09-28 23:18:53.713332: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:533] model_pruner failed: Invalid argument: MutableGraphView::MutableGraphView error: node 'loss_1/reshape_1_loss/mean_squared_error/weighted_loss/concat' has self cycle fanin 'loss_1/reshape_1_loss/mean_squared_error/weighted_loss/concat'.
2021-09-28 23:18:53.717997: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:533] remapper failed: Invalid argument: MutableGraphView::MutableGraphView error: node 'loss_1/reshape_1_loss/mean_squared_error/weighted_loss/concat' has self cycle fanin 'loss_1/reshape_1_los

8039/8039 [==============================] - 11s 1ms/step - loss: 0.2007 - mse: 0.2007 - val_loss: 0.2560 - val_mse: 0.2560
Epoch 2/5
8039/8039 [==============================] - 10s 1ms/step - loss: 0.1984 - mse: 0.1984 - val_loss: 0.2563 - val_mse: 0.2563
Epoch 3/5
8039/8039 [==============================] - 11s 1ms/step - loss: 0.1984 - mse: 0.1984 - val_loss: 0.2533 - val_mse: 0.2533
Epoch 4/5
8039/8039 [==============================] - 11s 1ms/step - loss: 0.1983 - mse: 0.1983 - val_loss: 0.2495 - val_mse: 0.2495
Epoch 5/5
10049/10049 [==============================] - 4s 432us/step


TadGAN is also a reconstruction based pipeline, thus we specify the `input_shape` to be of multivariate shape as needed.

```python3
hyperparameters = {
    'orion.primitives.tadgan.TadGAN#1': {
        'epochs': 5,
        'verbose': True,
        'input_shape': [100, 25]
    }
}
```